**EZMALL Recommendation System:**


---

By: 

Prem Jha(premjha3010@gmail.com)
  
Aditya Jain(adityajain1974@gmail.com)

Pranav Badhe(badhepranav5@gmail.com)

*This notebook contains model creation for the system*

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors

import pprint

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics.pairwise import euclidean_distances

In [2]:
df=pd.read_csv("/content/data_new.csv")

In [3]:
df

,Unnamed: 0,id,title,description,price,link,image_link,brand,color,shipping,size,custom_label_0,custom_label_1,custom_label_2,category
0,0,1,Set of 3 Rani Haar and Earrings by Touchstone,Online Shopping for Set of 3 Rani Haar and Ear...,1800.0,https://www.ezmall.com/women/jewellery/artific...,https://cdn.ezmall.com/0/images/product/1/001-...,Touchstone,MULTI,51.00 INR,no size,Women,Jewellery,Artificial Jewellery,Apparel & Accessories
1,1,5,Lotus Herbals 5 Pc Anti Ageing Kit,Online Shopping for Lotus Herbals 5 Pc Anti Ag...,1765.0,https://www.ezmall.com/women/beauty-personal-c...,https://cdn.ezmall.com/0/images/product/5/Lotu...,Lotus Herbals,"MULTI,GOLD",101.00 INR,no size,Women,Beauty & Personal Care,Body & Skin Care,Health & Beauty
2,2,27,Touchstone 6 Pair Ethnic Earring Collection,Online Shopping for Touchstone 6 Pair Ethnic E...,1800.0,https://www.ezmall.com/women/jewellery/artific...,https://cdn.ezmall.com/0/images/product/27/001...,Touchstone,no color,51.00 INR,no size,Women,Jewellery,Artificial Jewellery,Apparel & Accessories
3,3,47,Khadi Mauri 12 Pc. Herbal Skin Care Kit not use,Wondering if one kit alone could suffice the b...,1600.0,https://www.ezmall.com/women/beauty-personal-c...,https://cdn.ezmall.com/0/images/product/47/Kha...,Khadi Mauri,no color,51.00 INR,no size,Women,Beauty & Personal Care,Body & Skin Care,Health & Beauty
4,4,53,2 Rajasthani Bed Sheets & 4 Pillow Covers Not USE,Inspired from the rich architectural heritage ...,5000.0,https://www.ezmall.com/home-kitchen/furnishing...,https://cdn.ezmall.com/0/images/product/53/001...,Bella Casa,no color,101.00 INR,no size,Home & Kitchen,Furnishings & Furniture,Bed Linen,Home & Garden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,2125,141221,Pick Any 1 Wardrobe with Shoe Rack + Free 3 Pc...,Pick Any 1 Wardrobe with Shoe Rack + Free 3 Pc...,4999.0,https://www.ezmall.com/home-kitchen/home-impro...,https://cdn.ezmall.com/0/images/product/141221...,Harrisson,"PINK,PURPLE,BROWN",201.00 INR,no size,Home & Kitchen,Home Improvement,Home Storage & Organising,Home & Garden
2126,2126,141222,Pick any 1 Leather Retail Shoulder Handbag,Pick any 1 Leather Retail Shoulder Handbag,999.0,https://www.ezmall.com/women/women-accessories...,https://cdn.ezmall.com/0/images/product/141222...,Leather Retail,"MAROON,YELLOW,PINK,BLACK,PURPLE",0.00 INR,no size,Women,Women Accessories,Hand Bags & Clutches,Apparel & Accessories
2127,2127,141223,Pick Any 1 Leather Retail Women Beautiful Handbag,Pick Any 1 Leather Retail Women Beautiful Handbag,999.0,https://www.ezmall.com/women/women-accessories...,https://cdn.ezmall.com/0/images/product/141223...,Leather Retail,"MAROON,BLACK",0.00 INR,no size,Women,Women Accessories,Hand Bags & Clutches,Apparel & Accessories
2128,2128,141246,Egg Boiler,Egg Boiler,1000.0,https://www.ezmall.com/home-kitchen/kitchen-di...,https://cdn.ezmall.com/0/images/product/141246...,Unique Dealz,no color,0.00 INR,no size,Home & Kitchen,Kitchen & Dining,Kitchen Tools,Home & Garden


In [4]:
df=df.drop(df.columns[0], axis=1)

In [5]:
df.shape

(2130, 14)

In [6]:
df.columns

Index(['id', 'title', 'description', 'price', 'link', 'image_link', 'brand',
       'color', 'shipping', 'size', 'custom_label_0', 'custom_label_1',
       'custom_label_2', 'category'],
      dtype='object')

In [7]:
df['price']=df['price'].astype(int)

In [8]:
df.loc[ (df['price'] >= 0) & (df['price'] <= 1000), 'price_range'] = "0-1000"
df.loc[ (df['price'] > 1000) & (df['price'] <= 2000), 'price_range'] = "1000-2000"
df.loc[ (df['price'] > 2000) & (df['price'] <= 3000), 'price_range'] = "2000-3000"
df.loc[ (df['price'] > 3000) & (df['price'] <= 4000), 'price_range'] = "3000-4000"
df.loc[ (df['price'] > 4000) & (df['price'] <= 5000), 'price_range'] = "4000-5000"
df.loc[ (df['price'] > 5000) & (df['price'] <= 6000), 'price_range'] = "5000-6000"
df.loc[ (df['price'] > 6000) & (df['price'] <= 7000), 'price_range'] = "6000-7000"
df.loc[ (df['price'] > 7000) & (df['price'] <= 8000), 'price_range'] = "7000-8000"
df.loc[ (df['price'] > 8000) & (df['price'] <= 9000), 'price_range'] = "8000-9000"
df.loc[ (df['price'] > 9000) & (df['price'] <= 10000), 'price_range'] = "9000-10000"

In [9]:
df

,id,title,description,price,link,image_link,brand,color,shipping,size,custom_label_0,custom_label_1,custom_label_2,category,price_range
0,1,Set of 3 Rani Haar and Earrings by Touchstone,Online Shopping for Set of 3 Rani Haar and Ear...,1800,https://www.ezmall.com/women/jewellery/artific...,https://cdn.ezmall.com/0/images/product/1/001-...,Touchstone,MULTI,51.00 INR,no size,Women,Jewellery,Artificial Jewellery,Apparel & Accessories,1000-2000
1,5,Lotus Herbals 5 Pc Anti Ageing Kit,Online Shopping for Lotus Herbals 5 Pc Anti Ag...,1765,https://www.ezmall.com/women/beauty-personal-c...,https://cdn.ezmall.com/0/images/product/5/Lotu...,Lotus Herbals,"MULTI,GOLD",101.00 INR,no size,Women,Beauty & Personal Care,Body & Skin Care,Health & Beauty,1000-2000
2,27,Touchstone 6 Pair Ethnic Earring Collection,Online Shopping for Touchstone 6 Pair Ethnic E...,1800,https://www.ezmall.com/women/jewellery/artific...,https://cdn.ezmall.com/0/images/product/27/001...,Touchstone,no color,51.00 INR,no size,Women,Jewellery,Artificial Jewellery,Apparel & Accessories,1000-2000
3,47,Khadi Mauri 12 Pc. Herbal Skin Care Kit not use,Wondering if one kit alone could suffice the b...,1600,https://www.ezmall.com/women/beauty-personal-c...,https://cdn.ezmall.com/0/images/product/47/Kha...,Khadi Mauri,no color,51.00 INR,no size,Women,Beauty & Personal Care,Body & Skin Care,Health & Beauty,1000-2000
4,53,2 Rajasthani Bed Sheets & 4 Pillow Covers Not USE,Inspired from the rich architectural heritage ...,5000,https://www.ezmall.com/home-kitchen/furnishing...,https://cdn.ezmall.com/0/images/product/53/001...,Bella Casa,no color,101.00 INR,no size,Home & Kitchen,Furnishings & Furniture,Bed Linen,Home & Garden,4000-5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,141221,Pick Any 1 Wardrobe with Shoe Rack + Free 3 Pc...,Pick Any 1 Wardrobe with Shoe Rack + Free 3 Pc...,4999,https://www.ezmall.com/home-kitchen/home-impro...,https://cdn.ezmall.com/0/images/product/141221...,Harrisson,"PINK,PURPLE,BROWN",201.00 INR,no size,Home & Kitchen,Home Improvement,Home Storage & Organising,Home & Garden,4000-5000
2126,141222,Pick any 1 Leather Retail Shoulder Handbag,Pick any 1 Leather Retail Shoulder Handbag,999,https://www.ezmall.com/women/women-accessories...,https://cdn.ezmall.com/0/images/product/141222...,Leather Retail,"MAROON,YELLOW,PINK,BLACK,PURPLE",0.00 INR,no size,Women,Women Accessories,Hand Bags & Clutches,Apparel & Accessories,0-1000
2127,141223,Pick Any 1 Leather Retail Women Beautiful Handbag,Pick Any 1 Leather Retail Women Beautiful Handbag,999,https://www.ezmall.com/women/women-accessories...,https://cdn.ezmall.com/0/images/product/141223...,Leather Retail,"MAROON,BLACK",0.00 INR,no size,Women,Women Accessories,Hand Bags & Clutches,Apparel & Accessories,0-1000
2128,141246,Egg Boiler,Egg Boiler,1000,https://www.ezmall.com/home-kitchen/kitchen-di...,https://cdn.ezmall.com/0/images/product/141246...,Unique Dealz,no color,0.00 INR,no size,Home & Kitchen,Kitchen & Dining,Kitchen Tools,Home & Garden,0-1000


# K-nearest neighbour


In [10]:
df['title'][1930]

'Marbello Rich Ivory Stainless Steel Insulated Kitchen Set'

In [11]:
brand_df = pd.get_dummies(df['brand'])
category_df = pd.get_dummies(df['category'])
label0_df = pd.get_dummies(df['custom_label_0'])
label1_df = pd.get_dummies(df['custom_label_1'])
label2_df = pd.get_dummies(df['custom_label_2'])
price_df=pd.get_dummies(df['price_range'])

In [12]:
features_df = pd.concat([brand_df, category_df,label0_df,label1_df,label2_df,price_df], axis=1)

In [13]:
min_max_scaler = MinMaxScaler()
features = min_max_scaler.fit_transform(features_df)

In [14]:
features_df.to_csv('features.csv')

In [15]:
def knn(m):
  model = neighbors.NearestNeighbors(n_neighbors=m, algorithm='auto')
  model.fit(features) 
  dist, idlist = model.kneighbors(features)
  return dist,idlist

In [16]:
def Recommender(name,n):
    dist,idlist=knn(n+1)
    prod_list = []
    id_list=[]
    recomm=[]
    prod_id = df[df['title'] == name].index
    prod_id = prod_id[0]
    print(type(prod_id))
    for newid in idlist[prod_id]:
        prod_list.append(df.iloc[newid].title)
        id_list.append(newid)
    for i in range(0,len(prod_list)):
      recomm.append(f"{prod_list[i]}")
    return recomm

In [17]:
input="Blue Faded Whiskered Jeans"
nn=Recommender(input,7)

<class 'numpy.int64'>


In [18]:
if str(nn[0])==str(input):
      print(*nn[1:], sep="\n")
else:
      print(*nn, sep="\n")

Teal Blue Whiskered Jeans
Dark Indigo Jeans
Dark Blue Tinted Washed Jeans
Blue Faded Jeans
Light Blue Washed Jeans
Blue Rain Drop Whiskered Washed Jeans
Grey-Blue Basic Jeans


# Cosine similarity

In [19]:
def to_low(strok:str):
    return strok.lower()

In [20]:
df.columns

Index(['id', 'title', 'description', 'price', 'link', 'image_link', 'brand',
       'color', 'shipping', 'size', 'custom_label_0', 'custom_label_1',
       'custom_label_2', 'category', 'price_range'],
      dtype='object')

In [21]:
index_df = df[['title', 'brand', 'category','custom_label_0','custom_label_1','custom_label_2']].copy()

In [22]:
cols = ['category', 'brand','custom_label_0','custom_label_1','custom_label_2']
index_df['feature'] = index_df['title'].copy().apply(to_low)
for i in cols:
  index_df['feature'] += ' ' + index_df[i].apply(to_low)

In [23]:
index_df['custom_label_1'][111]

'Ethnic Wear'

In [24]:
#index_df.to_csv("index.csv")

In [25]:
index_df['feature']

0       set of 3 rani haar and earrings by touchstone ...
1       lotus herbals 5 pc anti ageing kit health & be...
2       touchstone 6 pair ethnic earring collection ap...
3       khadi mauri 12 pc. herbal skin care kit not us...
4       2 rajasthani bed sheets & 4 pillow covers not ...
                              ...                        
2125    pick any 1 wardrobe with shoe rack + free 3 pc...
2126    pick any 1 leather retail shoulder handbag app...
2127    pick any 1 leather retail women beautiful hand...
2128    egg boiler home & garden unique dealz home & k...
2129    7 pc knife set home & garden unique dealz home...
Name: feature, Length: 2130, dtype: object

In [26]:
count_vec = CountVectorizer(stop_words='english')
count_matrix = count_vec.fit_transform(index_df['feature'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [27]:
def cosine_recommender(nam:str,n):
    ind = index_df[index_df['title'] == nam].index.to_list()[0]
    cos_scor = list(enumerate(cosine_sim[ind]))
    cos_scor = sorted(cos_scor, key=lambda x: x[1], reverse=True)
    cos_scor = cos_scor[0:n+1]
    new_ind = [i[0] for i in cos_scor]
    return index_df['title'].iloc[new_ind]

In [28]:
prod_name="Marbello Rich Ivory Stainless Steel Insulated Kitchen Set"
cos=cosine_recommender(prod_name,9)
cos=cos.to_list()
if str(cos[0])==str(prod_name):
      print(*cos[1:], sep="\n")
else:
      print(*cos, sep="\n")

Modware Set of 3 Stainless Steel Insulated Casseroles
Set of 5 Stainless Steel Cookware Set
Set of 3 Stainless Steel Insulated Casserole
Electric Kettle with Stainless steel insulated 25 pcs kitchen Set
Kool Kammander Summer Set of 14 Kitchen Combo
5 Ltr. Stainless Steel Insulated Dispenser with 4 Steel Glasses
Set of 5 - 4 in 1 Storage Set
Set of 3 Stainless Steel Casseroles + 1 Stainless Steel Electric Kettle
Stainless Steel Insulated 3 Casseroles + 1 Flask + 5 Ltr.Dispenser + 1 Ladle & Free 6 Glass+Tray+Jar


# Combiner

In [37]:
def combiner(prod_name,n):
  prod=list() 
  cos=cosine_recommender(prod_name,50)
  cos=cos.to_list()
  neir=Recommender(prod_name,50)
  for j in range(0,len(neir)):
    for k in range(0,len(cos)):
      if neir[j]==cos[k]:
        prod.append(neir[j]) 
  return prod[0:n]

In [38]:
n=12
out=combiner('Marbello Rich Ivory Stainless Steel Insulated Kitchen Set',n)
for i in range(0,n):
  print(f"{i+1}){out[i]}")

<class 'numpy.int64'>
1)Marbello Rich Ivory Stainless Steel Insulated Kitchen Set
2)Set of 5 Stainless Steel Cookware Set
3)Stainless Steel Insulated 3 Casseroles + 1 Flask + 5 Ltr.Dispenser + 1 Ladle & Free 6 Glass+Tray+Jar
4)Stainless Steel Insulated 3 Casseroles + 3 Serving Spoons & Free 6 Bowls + 6 Glasses + 6 Tea Spoons
5)Pick Any 1 - 5 Pc Broad Size Non-Stick Cookware Set
6)Kool Kammander Summer Set of 14 Kitchen Combo
7)3 Insulated Casseroles + 1ltr.Flask + 4ltr.Dispenser + 6 Bowls +Ladle+Free 6 Glass, 6 Coasters,Tray
8)Modware Set of 3 Stainless Steel Insulated Casseroles
9)Shagun 9 Pcs Aluminium Cookware Set
10)9 Pcs Cook & Serve Handi & Patila Combo Lids
11)Set of 5 - 4 in 1 Storage Set
12)6 Pcs Non-Stick Cookware Set


# Experiment

In [31]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [32]:
from scipy.stats import pearsonr
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
stopwords_list = stopwords.words('english')
vectorizer = TfidfVectorizer(analyzer='word')
tfidf_matrix = vectorizer.fit_transform(df['title'])

In [34]:
tfidf_des_array = tfidf_matrix.toarray()

In [35]:
def recommend_pearson(nam,n):
    ind = index_df[index_df['title'] == nam].index.to_list()[0]
    correlation = []
    for i in range(len(tfidf_des_array)):
        correlation.append(pearsonr(tfidf_des_array[ind], tfidf_des_array[i])[0])
    correlation = list(enumerate(correlation))
    sorted_corr = sorted(correlation, reverse=True, key=lambda x: x[1])[1:n+1]
    ind = [i[0] for i in sorted_corr]
    return index_df['title'].iloc[ind]

In [36]:
prod_name="Ziraffe Brown Genuine Leather Brogues"
pear=recommend_pearson(prod_name,9)
pear=pear.to_list()
if str(pear[0])==str(prod_name):
      print(*pear[1:], sep="\n")
else:
      print(*pear, sep="\n")

Ziraffe Black Genuine Leather Brogues
Lawman Brown Brogues
Ziraffe Brown Oxford Shoes
Fabbro Men Genuine Leather Wallet - Brown - 10
Fabbro Men Genuine Leather Wallet - Brown-014
Fabbro Men Genuine Leather Wallet Brown - 047
Fabbro Casual Men Genuine Leather Wallet - Brown-011
Fabbro Men Genuine Leather Casual Belt - Brown-037
Fabbro Men Genuine Leather Casual Belt - Brown-031
